<h1>Superhero Exploratory Analysis and Research Proposal</h1>
<p>The purpose of this projet is to explore and analyze superheros and their characteristics.</p>
<p>Data Source: https://www.kaggle.com/claudiodavi/superhero-set</p>
<p>There are two datasets: The first contains characteristics of each superhero and the second lists the superpowers that each hero has.</p>
<p>This kernel is part of a project for Thinkful's Data Science Bootcamp.</p>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = [20.0, 7.0]
plt.rcParams.update({'font.size': 22})

sns.set_palette('bright')
sns.set_style('whitegrid')
sns.set_context('talk')

In [ ]:
info_df = pd.read_csv('../input/heroes_information.csv')
powers_df = pd.read_csv('../input/super_hero_powers.csv')

In [ ]:
print(info_df.shape)
info_df.head()

In [ ]:
info_df.columns

In [ ]:
#drop unnamed column
info_df = info_df.drop(['Unnamed: 0'], axis=1)

In [ ]:
#check for null values
info_df.isnull().sum()

In [ ]:
#show rows where publisher is null
info_df[info_df.Publisher.isnull()]

In [ ]:
# get rid of NaN values for publisher
info_df.Publisher.fillna('Other', inplace=True)

In [ ]:
# -99 is not a valid height or weight so set to NaN
info_df.replace(-99.0, np.nan, inplace=True)

In [ ]:
#update genders if -
info_df.Gender.replace('-', 'Non-Binary', inplace=True)

In [ ]:
#update alignment if -
info_df.Alignment.replace('-', 'None', inplace=True)

In [ ]:
info_df.head()

In [ ]:
#plot number of superheros by gender
sns.countplot(x='Gender', data=info_df);

plt.title('Superheros by Gender')
plt.ylabel('Number of Superheros')
plt.show();

Notice there are many more males than females and many more females than non-binary.

In [ ]:
#create new dataframe with only height and weight values that are not null
info_2 = info_df.dropna(subset=['Height', 'Weight'], how='any')
info_2.isnull().sum()

In [ ]:
#keep only columns of interest for plotting
info_2 = info_2[['name', 'Gender', 'Height', 'Weight', 'Alignment']]

In [ ]:
#plot height and weight by gender
sns.pairplot(data=info_2, hue='Gender', size=5)

plt.title('Superhero Heights and Weights by Gender')
plt.show();

In [ ]:
sns.distplot(info_2.Weight, kde=False)
plt.title('Histogram of Superhero Weight')
plt.show();

In [ ]:
#another height and weight visualization
sns.kdeplot(info_2['Weight'], info_2['Height'])

plt.title('Superheros Height and Weight')
plt.show();

In [ ]:
#plot alignment by gender
sns.countplot(info_2.Gender, hue=info_2.Alignment)

plt.title('Alignment by Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Heros')
plt.show();

In [ ]:
#neutral and none groups are both very small...combining into one group as neutral
info_2.Alignment.replace('None', 'neutral', inplace=True)
info_df.Alignment.replace('None', 'neutral', inplace=True)

In [ ]:
info_2.Gender.value_counts()

In [ ]:
info_2.Alignment.value_counts()

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(hspace=0.25, wspace=0.25)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

fig.add_subplot(221)
sns.swarmplot(x=info_2.Alignment, y=info_2.Weight, hue=info_2.Gender)

fig.add_subplot(222)
sns.stripplot(x=info_2.Alignment, y=info_2.Weight, hue=info_2.Gender, jitter=True)

fig.add_subplot(223)
sns.swarmplot(x=info_2.Gender, y=info_2.Weight, hue=info_2.Alignment)

fig.add_subplot(224)
sns.stripplot(x=info_2.Gender, y=info_2.Weight, hue=info_2.Alignment, jitter=True)

plt.title('Superhero Characteristics')
plt.show;

In [ ]:
#plot number of superheros per publisher
sns.countplot(x="Publisher", data=info_df)

plt.title('Number of Superheros by Publisher')
plt.ylabel('Number of Superheros')
plt.xlabel('Publisher')
plt.xticks(rotation = 90)
plt.show();

In [ ]:
#create new dataframe with only small publishers with < 200 superheros
counts = info_df.Publisher.value_counts()
small_publishers_df = info_df.loc[info_df['Publisher'].isin(counts[counts < 200].index), :]

small_publishers_count = small_publishers_df.Publisher.value_counts()

In [ ]:
#plot only the small publishers
sns.countplot(x="Publisher", data=small_publishers_df)

plt.title('Number of Superheros by Publisher')
plt.ylabel('Number of Superheros')
plt.xlabel('Publisher')
plt.xticks(rotation = 90)
plt.show();

In [ ]:
small_publishers_df.sort_values('Publisher')

In [ ]:
powers_df.head(10)

In [ ]:
# check if any NaN values
powers_df[powers_df.isnull().any(axis=1)]

In [ ]:
#create new column with sum of each superheros powers
powers_df['sum_powers'] = powers_df.iloc[2:].sum(axis=1)

In [ ]:
#sort by most powers
most_powers = powers_df.sort_values('sum_powers', ascending=False)
most_powers.head()

In [ ]:
#plot superheros by number of powers
sns.countplot(x='sum_powers', data=powers_df)

plt.xlabel('Number of Powers')
plt.ylabel('Number of Superheros')
plt.title('Superheros by Number of Powers')
plt.show();

In [ ]:
#plot superheros with the most powers
most_powers = powers_df[['hero_names', 'sum_powers']].sort_values('sum_powers', ascending=False)

sns.barplot(x=most_powers['hero_names'].head(20),
            y='sum_powers', 
            data=most_powers)

plt.xticks(rotation=80)
plt.title('Superheros with the Most Powers')
plt.xlabel('Superhero')
plt.ylabel('Number of Powers')
plt.show();

In [ ]:
#create a word cloud of superheros with the most powers
from wordcloud import WordCloud

#remove spaces in between name parts - wordcloud splits on whitespace
heronames = most_powers['hero_names'].str.replace(' ', '')

wordcloud = WordCloud().generate(str(heronames.head(20)))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#create new df with number of superheros with each power
sum_row = {col: powers_df[col].sum() for col in powers_df}

sum_df = pd.DataFrame(sum_row, index=["Total"])

sum_df = sum_df.drop('hero_names', axis=1)
sum_df = sum_df.drop('sum_powers', axis=1)


sum_df.tail()

In [ ]:
#transform df
sum_df = sum_df.T
sum_df.head()

In [ ]:
sum_df = sum_df.reset_index()
sum_df.rename(columns = {'index':'Powers'}, inplace=True)

In [ ]:
sum_df.columns

In [ ]:
#sort powers by most common
sum_df = sum_df.sort_values('Total', ascending=False)
sum_df.head()

In [ ]:
#plot most common superpowers
sns.barplot(x=sum_df['Powers'].head(20),
            y='Total', 
            data=sum_df)

plt.xticks(rotation=80)
plt.title('Most Common Superpowers')
plt.xlabel('Superpower')
plt.ylabel('Superheros with Power')
plt.show();

In [ ]:
#plot least common superpowers
sns.barplot(x=sum_df['Powers'].tail(20),
            y='Total', 
            data=sum_df)

plt.xticks(rotation=80)
plt.title('Least Common Superpowers')
plt.xlabel('Superpower')
plt.ylabel('Superheros with Power')
plt.show();

In [ ]:
#merge original dataframes together
merged_df = pd.merge(info_df, powers_df, left_on='name', right_on='hero_names', how='inner')

In [ ]:
#view new df head
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
#plot number of superpowers by publisher
sns.barplot(x='Publisher', y='sum_powers', data=merged_df)
plt.xticks(rotation = 90);

plt.title('Number of Powers per Superhero by Publisher')
plt.xlabel('Publisher')
plt.ylabel('Number of Powers')
plt.show();

<p>There seems to be a very large variation in superheros.  This leads to the question what exactly is a superhero?</p>

<p>According to <a href='https://www.merriam-webster.com/dictionary/superheroSuperhero' target='blank'>Merriam Webster</a> a superhero is fictional hero having extraordinary or superhuman powers or also an exceptionally skillful or successful person.</p>



In [ ]:
#What superhero is from South Park?
merged_df.loc[merged_df['Publisher'] == 'South Park']

In [ ]:
#Is he really a superhero?? where is Mario then!?
merged_df.loc[merged_df['Publisher'] == 'Microsoft']

In [ ]:
merged_df.Publisher.value_counts()

In [ ]:
#the small publishers have significantly less superheros so keeping only the superheros from Marvel and DC
merged_df = merged_df[(merged_df['Publisher'] == 'Marvel Comics') | (merged_df['Publisher'] == 'DC Comics')]

In [ ]:
merged_df.Publisher.value_counts()

In [ ]:
#t-test comparing number of male heroes to female
print(ttest_ind(merged_df['Gender'] == 'Male', merged_df['Gender'] == 'Female'))

In [ ]:
#t-test comparing number of powers for males and females
ttest_ind(merged_df.dropna()['sum_powers'][merged_df['Gender'] == 'Male'],
          merged_df.dropna()['sum_powers'][merged_df['Gender'] =='Female'])

In [ ]:
#find mean number of powers by gender
merged_df.groupby('Gender')['sum_powers'].mean()

In [ ]:
#plot number of superpowers by publisher
sns.barplot(x='Publisher', y='sum_powers', hue='Gender', data=merged_df)

plt.title('Number of Powers per Superhero by Publisher')
plt.xlabel('Publisher')
plt.ylabel('Number of Powers')
plt.show();

In [ ]:
#plot number of powers by Alignment
sns.boxplot(x=merged_df.Alignment, y=merged_df.sum_powers, hue=merged_df.Gender)

plt.title('Number of Powers per Superhero by Alignment')
plt.xlabel('Alignment')
plt.ylabel('Number of Powers')
plt.show();

<h2>Data Experiment Proposal</h2>

<p>Simply adding up the number of powers is not enough to determine the most powerful superhero.  We also need to take into account the value of each power.  The website, superherodb.com that this dataset was scraped from also has a power bonus for each available power.  Some powers are obviously more valueable than others and should be weighted more heavily.  Also, some heroes have weaknesses that should also be accounted for to find a true ranking of most powerful superhero.</p>

<p>My hypothesis is that once the relative power levels and weaknesses are accounted for, female superheroes are stronger than male superheroes eventhough they are generally given a lower number of powers than male superheroes.</p>
    

<h4>Rollout Plan</h4>

<p>
Write a scraper to collect more data from superherodb.com, making sure to include the power bonus values for each power and any weaknesses each superhero may have.  After collecting the additional information, explore the data and calculate statistics.
</p>

<h4>Evaluation Plan</h4>

<p>
Rank the superheroes by the relative power levels, and not just by the total number of powers.  Find the mean power level and standard deviation for each group of heroes, male and female, and compare the two groups.  And finally compare the males to females to determine the most powerful gender.
</p>